In [14]:
import numpy as np
import matplotlib.pyplot as plt
def fonction_g_discrete(choix_angle,v):
    if v==0:
        print("ici")
        x=0
        y=0
    else: 
        if choix_angle==1:
            x=1
            y=0
        if choix_angle==2:
            x=-1
            y=0
        if choix_angle==3:
            x=0
            y=1
        if choix_angle==4:
            x=0
            y=-1
    return([x,y])



In [20]:
import random
import numpy as np
import pandas as pd

def overall():
    xcible=5
    ycible=5
    lim_x=[0,20]
    lim_y=[0,20]
    destination=[xcible,ycible]
    Liste_realisations=[]
    table_cout=np.random.multivariate_normal(size=20,mean=[-5]*20,cov=np.eye(20))
    goal_x=5
    goal_y=5
    for J in range(30):
        T=np.random.choice([20,50,100])
        Table_nom=[J]*T
        Recompenses=[]
        Actions=[]
        Nouvel_etat=[]
        Positions_t=[]
        x_si=np.random.choice(range(20))
        y_si=np.random.choice(range(20))
        situation_initiale=np.array([x_si,y_si])
        for j in range(T):
            vitesse=np.random.choice([0,1,2,3])
            if situation_initiale[0]==goal_x and situation_initiale[1]==goal_y:
                recompense=1
            else:
                recompense=-1
            if vitesse!=0:

                choix_angle=random.choice([1,2,3,4])
                changement_pos=fonction_g_discrete(choix_angle=choix_angle,v=vitesse)
                situation_nouvelle=np.add(changement_pos,situation_initiale)
                if situation_nouvelle[0]>lim_x[0] or situation_nouvelle[0]>lim_x[1]:
                    recompense=+-20*vitesse
                    situation_nouvelle=situation_initiale
                elif situation_nouvelle[1]>lim_y[0] or situation_nouvelle[1]>lim_y[1]:
                    recompense=+-20*vitesse
                    situation_nouvelle=situation_initiale
                elif situation_nouvelle[0]==goal_x and situation_nouvelle[1]==goal_y:
                    recompense=1
                else:
                    recompense=+table_cout[situation_nouvelle[0],situation_nouvelle[1]]/vitesse
            else:
                situation_nouvelle=situation_initiale
                choix_angle=random.choice([0,1,2,3,4])
            Recompenses.append(recompense)
            Actions.append((vitesse,choix_angle))
            Positions_t.append(situation_initiale)
            Nouvel_etat.append(situation_nouvelle)
            situation_initiale=situation_nouvelle
        dico=pd.DataFrame({"recompense":Recompenses,"actions":Actions,"etat_t":Positions_t,"etat_suivant":Nouvel_etat,"Indice_simul":Table_nom})
        Liste_realisations.append(dico)
    return(pd.concat(Liste_realisations))

In [21]:
R=overall()

In [22]:
R

,recompense,actions,etat_t,etat_suivant,Indice_simul
0,-40,"(2, 3)","[10, 13]","[10, 13]",0
1,-40,"(2, 4)","[10, 13]","[10, 13]",0
2,-60,"(3, 4)","[10, 13]","[10, 13]",0
3,-40,"(2, 1)","[10, 13]","[10, 13]",0
4,-40,"(2, 3)","[10, 13]","[10, 13]",0
...,...,...,...,...,...
95,-60,"(3, 3)","[19, 16]","[19, 16]",29
96,-20,"(1, 3)","[19, 16]","[19, 16]",29
97,-40,"(2, 3)","[19, 16]","[19, 16]",29
98,-40,"(2, 3)","[19, 16]","[19, 16]",29
